# 1.Install lib

In [1]:
!pip install -U sentence-transformers

In [2]:
!pip install pymorphy2

In [3]:
!pip install emoji

In [4]:
!pip install contractions

In [5]:
#! pip install deeppavlov

In [6]:
#! pip install TorchCRF

In [7]:
#! pip install pytorch-crf

In [8]:
#! pip install natasha

In [9]:
#! python -m spacy download ru_core_news_sm

In [10]:
#! python -m spacy download ru_core_news_md

In [11]:
! python -m spacy download ru_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 1.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# 2.Import

In [12]:
import pandas as pd
import numpy as np
import scipy

import re
import torch
import pymorphy2
import re
import emoji
import unicodedata
import contractions
import inflect
import string
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from pymorphy2 import MorphAnalyzer

from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import random_projection
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_distances

#from deeppavlov import build_model, configs


import spacy
from spacy import load
from spacy.lang.ru.examples import sentences
from spacy.lang.ru import Russian
from spacy import displacy

In [13]:
# from natasha import Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger,NamesExtractor, Doc
# from natasha import MorphVocab, Doc
# from razdel import tokenize, sentenize

In [14]:
# import torch
# import torchcrf

In [15]:
#

# 3.Load Data

In [16]:
url = 'https://raw.githubusercontent.com/MaximkaB/Task/refs/heads/main/test_task/sber_security/text_raw.txt'

In [17]:
df=pd.read_table(url ,header=None, names=['news'] )

In [18]:
df.head(2)

,news
0,17 февраля 1993 Постановлением Правительства Р...
1,6 января 1993 Распоряжением Президента РФ Пред...


In [19]:
list_df=list(df['news'])

# 4.Preproceccing

In [20]:
df.head(2)


,news
0,17 февраля 1993 Постановлением Правительства Р...
1,6 января 1993 Распоряжением Президента РФ Пред...


In [21]:
# Создаем поле, где удаляем первые три сивповл дата-месяц-год
df['news_cor']=''
for i in range(len(df)):
  df.loc[i,'news_cor'] = ' '.join(df['news'].iloc[i].split()[3:])

In [22]:
df.head(2)

,news,news_cor
0,17 февраля 1993 Постановлением Правительства Р...,Постановлением Правительства РФ во исполнение ...
1,6 января 1993 Распоряжением Президента РФ Пред...,Распоряжением Президента РФ Председателем Сове...


## Approach 1

In [23]:
# Первый подход, удаление стоп слов, пунктуации
# Data Frame первого подхода df1

In [24]:
# Загрузка стоп-слов и пунктуации
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('russian'))
stop_words.add('«')
stop_words.add('»')
punctuation = set(string.punctuation)

def preprocess_text(text):
    #words = word_tokenize(text.lower())  # Привести к нижнему регистру и токенизировать
    words = word_tokenize(text)
    filtered_words = [word for word in words if word not in stop_words and word not in punctuation]
    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
df1=df.copy()

In [26]:
df1['news_trnsf1']=df1['news_cor'].apply(preprocess_text)

In [27]:
df1.head(2)

,news,news_cor,news_trnsf1
0,17 февраля 1993 Постановлением Правительства Р...,Постановлением Правительства РФ во исполнение ...,Постановлением Правительства РФ исполнение Ука...
1,6 января 1993 Распоряжением Президента РФ Пред...,Распоряжением Президента РФ Председателем Сове...,Распоряжением Президента РФ Председателем Сове...


## Approach 2

In [28]:
# Первый подход, здесь стоп слоова, пунктуацию, лемметизируем слова, оставляем только буквы
# Data Frame первого подхода df2

In [29]:
def preprocess_text(text):
    text = text.lower()
    text = text.replace('\r', ' ').replace('\n', ' ')
    text  = only_valid_symb(text)
    norm_text = only_valid_forms(text)
    return norm_text


def only_valid_symb(text: str)-> str:
    valid = set('йцукенгшщзхъфывапролджэячсмитьбюё ')
    txt =  ''.join(x for x in text if x in valid)
    while '  ' in txt:
        txt = txt.replace('  ', ' ')
    return txt.strip()

def only_valid_forms(text)->str:
    filter_forms = {'ADJF', 'ADJS', 'COMP', 'NUMR', 'NPRO', 'PREP', 'CONJ', 'PRCL', 'INTJ'}
    lst = []
    for word in text.split():
        tag = morph_parse(word)
        if any([x in tag.tag for x in filter_forms]):
            continue
        lst.append(tag.normal_form)
    return ' '.join(lst)

def morph_parse(word:str)->str:
    if word not in morph_dct:
        tag = morph.parse(word)[0]
        morph_dct[word] = tag
    return morph_dct[word]

In [30]:
df2=df.copy()
list_df2=list(df2['news'])
df2.head(2)

,news,news_cor
0,17 февраля 1993 Постановлением Правительства Р...,Постановлением Правительства РФ во исполнение ...
1,6 января 1993 Распоряжением Президента РФ Пред...,Распоряжением Президента РФ Председателем Сове...


In [31]:
morph=pymorphy2.MorphAnalyzer()
morph_dct={}

In [32]:
%%time
#f  =open(_SOURCE, 'rb')
data2 = list(df['news']) #list_df2 #pickle.load(f)
text2 = [preprocess_text(list_df2[i]) for i in range(len(list_df2))]
len(text2)

CPU times: user 212 ms, sys: 841 µs, total: 213 ms
Wall time: 214 ms


113

In [33]:
text2

['февраль постановление правительство рф исполнение указ президент рф концерн газпром преобразовать общество',
 'январь распоряжение президент рф председатель совет директор председатель правление рао газпром назначить рем вяхирев занимать пост председатель совет директор май год',
 'апрель начаться приватизация газпром',
 'май состояться собрание акционер газпром',
 'август состояться презентация офис газпром москва улица наметкина далее',
 'октябрь беларусь сварить стык газопровод ямал европа',
 'декабрь подписать соглашение поставка газ турция дать старт проект поток газопровод россия',
 'июнь решение собрание акционер рао газпром преобразовать общество',
 'июнь председатель совет директор оао газпром избрать виктор черномырдин занимать пост июнь год',
 'ноябрь начаться поставка газ газопровод ямал европа построить территория россия беларусь польша германия',
 'февраль край сварить стык газопровод поток',
 'июнь председатель совет директор газпром избрать дмитрий медведев занимать п

In [34]:
# удаление 1ого слова, в строке - это название месяца
df2['news_trnsf2']=''
for i in range(len(df2)):
  df2.loc[i,'news_trnsf2'] = ' '.join(text2[i].split()[1:])  #' '.join(df2_tmp['news'].iloc[i].split()[1:])


In [35]:
df2.head(2)

,news,news_cor,news_trnsf2
0,17 февраля 1993 Постановлением Правительства Р...,Постановлением Правительства РФ во исполнение ...,постановление правительство рф исполнение указ...
1,6 января 1993 Распоряжением Президента РФ Пред...,Распоряжением Президента РФ Председателем Сове...,распоряжение президент рф председатель совет д...


# 5.Baseline TF -IDF

In [36]:
# Выбираем с какой подходом предпроцессинга будем запускать tf-idf
# Работает лучше со 2ым

# df_data=df1.copy()
# var_news='news_trnsf1'

df_data=df2.copy()
var_news='news_trnsf2'

In [37]:
list_df_data=list(df_data[var_news])
text=df_data.copy()

In [38]:
# Создание объекта TfidfVectorizer
vectorizer = TfidfVectorizer()

# Преобразование текстов в TF-IDF матрицу
tfidf_matrix = vectorizer.fit_transform(df_data[var_news])

# Преобразование матрицы в DataFrame для удобства
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_df

,австрия,адрес,азия,актив,акционер,акция,алексей,арктика,армения,армросгазпром,...,электроэнергия,энергобезопасность,энергоблок,энергогенерация,энергоснабжение,этансодержать,юань,якутия,ямал,япония
0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.559789,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109,0.0,0.0,0.0,0.0,0.199213,0.199213,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
110,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
tfidf_df.shape

(113, 400)

In [40]:
tfidf_df_arr=np.array(tfidf_df)
tfidf_df_arr.shape

(113, 400)

In [41]:

# задаем номер новости для которой будем искать топ похожих
num=31

# задаем кол-во в топ выдаче похожих новостей
number_top_matches =3


sentences_full=list(df_data[var_news])

# если не задан номер новости для которой будем искать похожие, то берем данный номер случайную
if num== None:
  num=np.random.randint(len(sentences_full))


query = sentences_full[num]
queries = [query]
query_orig=list_df[num]
query_embeddings = np.array(tfidf_df.iloc[num]).reshape(1,len(tfidf_df.columns)) #model.encode(queries)

sentence_embeddings_base = np.array(tfidf_df) #model.encode(sentences)



print('num = ' , num)
print('query = ' , query)


num =  31
query =  сахалин запустить россия завод производство сжидить газ


In [42]:
query_embeddings.shape,sentence_embeddings_base.shape

((1, 400), (113, 400))

In [43]:
# Нахождение топ новостей методов TF - IDF
print("Semantic Search Results")

for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings_base, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    print("Query:", query_orig)
    print("\nTop 3 most similar sentences in corpus:")

    for idx, distance in results[1:number_top_matches+1]:
        print(list_df[idx].strip(), "(Cosine Score: %.4f)" % (1-distance))

Semantic Search Results
Query: 18 февраля 2009 На Сахалине запущен первый в России завод по производству сжиженного природного газа.

Top 3 most similar sentences in corpus:
9 июня 2021 Амурский газоперерабатывающий завод запущен в работу. (Cosine Score: 0.4355)
21 декабря 2006 «Газпром» стал участником «Сахалина-2» — первого в России проекта по производству СПГ. (Cosine Score: 0.3964)
14 октября 2015 Началось строительство Амурского газоперерабатывающего завода, самого мощного в России и второго по мощности в мире. (Cosine Score: 0.2571)


# 6.By Nerual Embedding

In [44]:
# находим эмбеддинги новостей, и по пим ищем топ похожих
# эмбеддинги рассчитываем на оснвое моделей name_model

In [45]:
name_model='DeepPavlov/rubert-base-cased-sentence'
#name_model='cointegrated/rubert-tiny2'
# name_model='sergeyzh/rubert-tiny-turbo'


model = SentenceTransformer(name_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [46]:
def nerual_similar(data,var_news,number_top_matches, num=None):

  """

  Описание:
    Функция выводит новость и топ number_top_matches похожих на нее

  Параметры:
    data - pandas дата фрейм в столбце которого загружена новости
    var_news - название столбца в котором загружены новости
    number_top_matches - кол - во похожих новостей для выбора
    num - номер новости для которой будем искать похожих.
          Если не задан, то будет выбираться случайная

  """

  # преобразуем в массив новости
  sentences=list(data[var_news])

  # выбираем номер новости для которой будем находить похожих
  if num == None:
    num=np.random.randint(len(sentences))

  # выбираем саму новость из преобразованных данных для которой будем находить похожих
  query = sentences[num]

  # выбираем саму новость из НЕ преобразованных данных для которой будем находить похожих
  query_orig=list_df[num]
  queries = [query]

  # находим эмбеддинг
  query_embeddings = model.encode(queries)

  #sentences=sentences_full.copy()

  sentence_embeddings_base = model.encode(sentences)

  print('Номер новости = ', num)

  print("Semantic Search Results")

  for query, query_embedding in zip(queries, query_embeddings):
      distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings_base, "cosine")[0]

      results = zip(range(len(distances)), distances)
      results = sorted(results, key=lambda x: x[1])
      print("Query:", query_orig)
      print("\nTop 3 most similar sentences in corpus:")

      for idx, distance in results[1:number_top_matches+1]:
          print(list_df[idx].strip(), "(Cosine Score: %.4f)" % (1-distance))

In [47]:
# Резульатат без обработки

nerual_similar(data=df1,
               var_news='news',
               number_top_matches=3,
               num=10
               )

Номер новости =  10
Semantic Search Results
Query: 3 февраля 2000 В Краснодарском крае сварен первый стык газопровода «Голубой поток».

Top 3 most similar sentences in corpus:
23 октября 1996 В Беларуси сварен первый стык газопровода «Ямал — Европа». (Cosine Score: 0.9143)
6 июня 2011 Введен в эксплуатацию газопровод «Джубга — Лазаревское — Сочи» — первый морской газопровод в России. (Cosine Score: 0.8972)
27 октября 2015 Сварен первый стык магистрального газопровода «Ухта — Торжок — 2». (Cosine Score: 0.8844)


In [48]:
# Резульатат при обраотки текста 1 подходом

nerual_similar(data=df1,
               var_news='news_trnsf1',
               number_top_matches=3,
               num=10
               )

Номер новости =  10
Semantic Search Results
Query: 3 февраля 2000 В Краснодарском крае сварен первый стык газопровода «Голубой поток».

Top 3 most similar sentences in corpus:
23 октября 1996 В Беларуси сварен первый стык газопровода «Ямал — Европа». (Cosine Score: 0.9012)
24 мая 2007 Завершено строительство первой очереди ПХГ «Хайдах» в Австрии — первого объекта подземного хранения газа в Европе, созданного при участии «Газпрома». (Cosine Score: 0.8856)
3 декабря 2008 Началась реализация мегапроекта «Ямал» — сварен первый стык системы магистральных газопроводов «Бованенково — Ухта», для транспортировки газа Бованенковского месторождения — крупнейшего на полуострове Ямал. (Cosine Score: 0.8780)


In [49]:
# Резульатат при обраотки текста 2 подходом

nerual_similar(data=df2,
               var_news='news_trnsf2',
               number_top_matches=3,
               num=10
               )

Номер новости =  10
Semantic Search Results
Query: 3 февраля 2000 В Краснодарском крае сварен первый стык газопровода «Голубой поток».

Top 3 most similar sentences in corpus:
27 октября 2015 Сварен первый стык магистрального газопровода «Ухта — Торжок — 2». (Cosine Score: 0.9578)
3 декабря 2008 Началась реализация мегапроекта «Ямал» — сварен первый стык системы магистральных газопроводов «Бованенково — Ухта», для транспортировки газа Бованенковского месторождения — крупнейшего на полуострове Ямал. (Cosine Score: 0.9115)
19 ноября 2018 Завершена морская укладка газопровода «Турецкий поток». (Cosine Score: 0.9060)


# 7.By Combo NER and nerual embedding

In [50]:
# Идея соеденить поиск NER и эмбеддинги
# 1. Ищем в каждой новости NER.
# 2. Находим пересечения NER целевой новости со всеми остальными
# 3. Отбиараем новости у которых есть пересечения по мно-ву NER с целевой
# 4. Из п.3 выбираем топ 3 (или меньше, если новосетей с пересечением по NER с целевой меньше 3)
# 5. Если в п.4 отобрали менее 3 новостей, то оставшиеся подбираем через ембеддинги по п.6

# NER - искал просто, смотрел какие есть название между символами "«" и "»"
# Пробывал искать NER через библиотеку spacу результат показался хуже, библиотека как раз не определяла как NER важные названия 'Северный поток' и т.п.

In [51]:
# копируем датафрейм, где есть новости перобразованные препроцессингом Approach 1
df_ner=df1.copy()
df_ner.head(2)

,news,news_cor,news_trnsf1
0,17 февраля 1993 Постановлением Правительства Р...,Постановлением Правительства РФ во исполнение ...,Постановлением Правительства РФ исполнение Ука...
1,6 января 1993 Распоряжением Президента РФ Пред...,Распоряжением Президента РФ Председателем Сове...,Распоряжением Президента РФ Председателем Сове...


In [52]:
# загрузка модели для поиска NER значений и леммитизации
ru_model = spacy.load('ru_core_news_lg')
#ru_model = spacy.load("ru_core_news_md")
#ru_model = spacy.load('ru_core_news_sm')

In [53]:
# загрузка модели дляпоиска эмбеддингов
name_model2='DeepPavlov/rubert-base-cased-sentence'
#name_model2='cointegrated/rubert-tiny2'
# name_model2='sergeyzh/rubert-tiny-turbo'


model2 = SentenceTransformer(name_model2)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [54]:
def nerual_similar_to_list(data, var_news, var_new_orig, number_top_matches, model, num=None):

  """

  Описание:
    Функция выводит новость и топ number_top_matches похожих на нее

  Параметры:
    data - pandas дата фрейм в столбце которого загружена новости
    var_news - название столбца в котором загружены новости
    number_top_matches - кол - во похожих новостей для выбора
    num - номер новости для которой будем искать похожих.
          Если не задан, то будет выбираться случайная
    model - переменная в которой лежит модель для поиска эмбеддингов

  """

  # преобразуем в массив новости
  sentences=list(data[var_news])

  # выбираем номер новости для которой будем находить похожих
  if num == None:
    num=np.random.randint(len(sentences))

  # выбираем саму новость из преобразованных данных для которой будем находить похожих
  query = sentences[num]

  # выбираем саму новость из НЕ преобразованных данных для которой будем находить похожих
  query_orig=list_df[num]
  queries = [query]

  # находим эмбеддинг
  query_embeddings = model.encode(queries)

  #sentences=sentences_full.copy()

  sentence_embeddings_base = model.encode(sentences)

  # print('Номер новости = ', num)

  # print("Semantic Search Results")

  #lst_out=[]

  for query, query_embedding in zip(queries, query_embeddings):
      distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings_base, "cosine")[0]

      results = zip(range(len(distances)), distances)
      results = sorted(results, key=lambda x: x[1])
      lst_out_ind=[e[0] for e in results[1:number_top_matches+1]]
      lst_out=list(data.loc[lst_out_ind,var_new_orig])

      return lst_out

In [55]:
def find_substrings_between_symbols(text, start_symbol, end_symbol):



# функция находит все подстроки, заключенные между двумя заданными символами

# Объяснение:
# Функция принимает три параметра: текст, символ начала и символ конца.
# С помощью метода find мы ищем индексы начала и конца подстрок.
# Если оба символа найдены, извлекаем подстроку между ними и добавляем её в список.
# Цикл продолжается до тех пор, пока не будут найдены все подстроки.
# В конце функция возвращает список найденных подстрок.

    substrings = []
    start_index = 0
    while True:
        # Находим индекс первого символа
        start_index = text.find(start_symbol, start_index)
        if start_index == -1:
            break  # Если символ не найден, выходим из цикла
        # Находим индекс второго символа
        end_index = text.find(end_symbol, start_index + 1)
        if end_index == -1:
            break  # Если символ не найден, выходим из цикла
        # Извлекаем подстроку и добавляем её в список
        substring = text[start_index + 1:end_index]
        substrings.append(substring)
        # Обновляем индекс для следующего поиска
        start_index = end_index + 1
    return substrings


In [56]:
def calc_ner_dist(news):

# '''

# Функция вычисляет из строки массив лемметизированных NER значений без дубликатов

# news - текстовая строка

# '''

  news=news.replace("РФ ", "")
  news=news.replace("Россия ", "")
  news=news.replace("Москва ", "")
  news=news.replace("Газпром ", "")

  # получаем множество NER значений и лемметизируем их
  doc=ru_model(news)
  new_ntr_set=set()

  # здесь ипользуюется поиск NER через заруженную модель, но с ним результата был хуже, закомментил его
  # for e in doc.ents:
  #   new_ntr_set.add(e.lemma_)


  text =doc.text
  start_symbol = '«'
  end_symbol = '»'
  result = find_substrings_between_symbols(text, start_symbol, end_symbol)

  result= [s.strip() for s in result]
  new_ntr_set_tmp=set()

  for e in result:
    doc_tmp=ru_model(e)

    lst_tmp=[token.lemma_ for token in doc_tmp]
    lst_tmp_s= " ".join(lst_tmp)
    new_ntr_set_tmp.add(lst_tmp_s)

  new_ntr_set=new_ntr_set | new_ntr_set_tmp

# удаляем часто встрчающиеся NER
  if 'газпром' in new_ntr_set:
    new_ntr_set.remove('газпром')

  if 'россия' in new_ntr_set:
    new_ntr_set.remove('россия')

  if 'москва' in new_ntr_set:
    new_ntr_set.remove('москва')

  if '«' in new_ntr_set:
    new_ntr_set.remove('«')

  if '»' in new_ntr_set:
    new_ntr_set.remove('»')


  return new_ntr_set

In [57]:
df_ner['ntr_set']=df_ner['news'].apply(calc_ner_dist)

In [58]:
df_ner.head(20)

,news,news_cor,news_trnsf1,ntr_set
0,17 февраля 1993 Постановлением Правительства Р...,Постановлением Правительства РФ во исполнение ...,Постановлением Правительства РФ исполнение Ука...,{}
1,6 января 1993 Распоряжением Президента РФ Пред...,Распоряжением Президента РФ Председателем Сове...,Распоряжением Президента РФ Председателем Сове...,{}
2,апрель 1994 Началась приватизация «Газпрома».,приватизация «Газпрома».,приватизация Газпрома,{}
3,31 мая 1995 Состоялось первое годовое Общее со...,Состоялось первое годовое Общее собрание акцио...,Состоялось первое годовое Общее собрание акцио...,{}
4,август 1995 Состоялась официальная презентация...,официальная презентация центрального офиса «Га...,официальная презентация центрального офиса Газ...,{}
5,23 октября 1996 В Беларуси сварен первый стык ...,В Беларуси сварен первый стык газопровода «Яма...,В Беларуси сварен первый стык газопровода Ямал...,{ямал — европа}
6,15 декабря 1997 Подписано межправительственное...,Подписано межправительственное соглашение о по...,Подписано межправительственное соглашение пост...,{голубой поток}
7,26 июня 1998 Решением собрания акционеров РАО ...,Решением собрания акционеров РАО «Газпром» пре...,Решением собрания акционеров РАО Газпром преоб...,{}
8,27 июня 1999 Председателем Совета директоров О...,Председателем Совета директоров ОАО «Газпром» ...,Председателем Совета директоров ОАО Газпром из...,{}
9,ноябрь 1999 Начались коммерческие поставки газ...,коммерческие поставки газа по газопроводу «Яма...,коммерческие поставки газа газопроводу Ямал — ...,{ямал — европа}


In [59]:
df_fun=df_ner.copy()

In [60]:
df_fun.head(2)

,news,news_cor,news_trnsf1,ntr_set
0,17 февраля 1993 Постановлением Правительства Р...,Постановлением Правительства РФ во исполнение ...,Постановлением Правительства РФ исполнение Ука...,{}
1,6 января 1993 Распоряжением Президента РФ Пред...,Распоряжением Президента РФ Председателем Сове...,Распоряжением Президента РФ Председателем Сове...,{}


In [61]:

# Здесь получаем в
# Параметры
# Датафрейм с оригинальными новостями и столбцом преобразованных новостей, по которому будем считать эмбеддинги
df_fun=df_ner.copy()
# Кол-во топ выдачи
number_top_matches=3
# номер новости для которой будем искать топ выдачу похожих
num=10
# назавание поля с новостями оригинал без преобразования
var_new_orig='news'

# названия поля с новостями по котрым будем искать эбмеддинги
var_news='news_trnsf1'
# название поля с множество NER(уже лемметизировнных) для новости var_new_orig
var_ner_set='ntr_set'




# сам новость к которой ищем похожие
query_orig=str(df_fun[var_new_orig][num])

df_fun['cnt_intercept']=0
df_fun['len_ner_set']=df_fun[var_ner_set].apply(len)

# NER самой новости к которой ищем похожие
ner_set=df_fun[var_ner_set][num]
df_fun['cnt_ner_query']=len(ner_set)
cnt_int=0

for i in range(len(df_fun)):
  if i != num:
    iter_set=df_fun[var_ner_set][i]
    set_int_tmp=ner_set & iter_set

  df_fun.loc[i,'cnt_intercept']=len(set_int_tmp)


df_fun['select_news_flg']= np.where(df_fun['cnt_intercept'] > 0,1,0)

# находим новости у которых были совпадние по NER с оригинальной новостью
df_fun_f=df_fun[df_fun['select_news_flg']==1]
len_tmp=len(df_fun_f)
# находим скольок берем новостей из совпадающих по NER, чтобы не перевалить за топ выдачу
min_s=min(number_top_matches,len_tmp)

# отбираем случайно из совпадаюющих по NER
df_fun_f_s=df_fun_f.sort_values('cnt_intercept',ascending=False)[:number_top_matches] # sample(min_s)

# массив где будет записаны топ новости но NER совпадению
news_list_ner=list(df_fun_f_s[var_new_orig])

# считаем скольок осталось вывести новостей,за исключением стольких, по скольким нашлись совпадения по NER
number_top_matches_rest=number_top_matches-len(news_list_ner)

# массив, где будут записыны топ новости по близости эмбеддингов к требуемой
news_lst_emb=[]

# Смоторим, если осталось мество в топ выдаче находим оставшиеся через эбмеддинги модели name_model(rubert)
if len(news_list_ner) < number_top_matches:

  #
  news_lst_emb=nerual_similar_to_list(data=df_fun,
              var_news=var_news,
              var_new_orig=var_new_orig,
               number_top_matches=number_top_matches_rest,
               model=model2,
               num=num,
               )
  news_list_fin=news_list_ner+news_lst_emb

  print("Query:", query_orig)
  print("\nTop 3 most similar sentences in corpus:")

  for e in news_list_fin:
    print(e)

else:
  print("Query:", query_orig)
  print("\nTop 3 most similar sentences in corpus:")

  for e in news_list_fin:
    print(e)


Query: 3 февраля 2000 В Краснодарском крае сварен первый стык газопровода «Голубой поток».

Top 3 most similar sentences in corpus:
15 декабря 1997 Подписано межправительственное соглашение о поставках российского газа в Турцию, давшее старт проекту «Голубой поток» — первому экспортному морскому газопроводу из России.
20 февраля 2003 Начались коммерческие поставки газа по «Голубому потоку».
23 октября 1996 В Беларуси сварен первый стык газопровода «Ямал — Европа».


In [62]:
print('Orig', query_orig)
print('\n')
for e in news_list_ner:
  print('By NER', e)
for e in news_lst_emb:
  print('By EMB', e)

Orig 3 февраля 2000 В Краснодарском крае сварен первый стык газопровода «Голубой поток».


By NER 15 декабря 1997 Подписано межправительственное соглашение о поставках российского газа в Турцию, давшее старт проекту «Голубой поток» — первому экспортному морскому газопроводу из России.
By NER 20 февраля 2003 Начались коммерческие поставки газа по «Голубому потоку».
By EMB 23 октября 1996 В Беларуси сварен первый стык газопровода «Ямал — Европа».


In [ ]:
# def calc_ner_dist(news):

# # '''

# # Функция вычисляет из строки массив лемметизированных NER значений без дубликатов

# # news - текстовая строка

# # '''

#   news=news.replace("РФ ", "")
#   news=news.replace("Россия ", "")
#   news=news.replace("Москва ", "")
#   # news=news.replace("«", "")
#   # news=news.replace("»", "")
#   news=news.replace("Газпром ", "")
#   # news=news.replace("«", "Имя ")

#   # получаем множество NER значений и лемметизируем их
#   doc=ru_model(news)
#   new_ntr_set=set()

#   for e in doc.ents:
#     new_ntr_set.add(e.lemma_)


#     text =doc.text
#     start_symbol = '«'
#     end_symbol = '»'
#     result = find_substrings_between_symbols(text, start_symbol, end_symbol)

#     result= [s.strip() for s in result]
#     new_ntr_set_tmp=set()

#     for e in result:
#       doc_tmp=ru_model(e)

#       lst_tmp=[token.lemma_ for token in doc_tmp]
#       lst_tmp_s= " ".join(lst_tmp)
#       new_ntr_set_tmp.add(lst_tmp_s)

#     new_ntr_set=new_ntr_set | new_ntr_set_tmp

# # удаляем часто встрчающиеся NER
#     if 'газпром' in new_ntr_set:
#       new_ntr_set.remove('газпром')

#     if 'россия' in new_ntr_set:
#       new_ntr_set.remove('россия')

#     if 'москва' in new_ntr_set:
#       new_ntr_set.remove('москва')

#     if '«' in new_ntr_set:
#       new_ntr_set.remove('«')

#     if '»' in new_ntr_set:
#       new_ntr_set.remove('»')


#   return new_ntr_set